In [1]:
import volta

In this example file, we want to compare two gene networks (for example one representing a control and the other a treated sample; when using the example data two networks treated by different drugs are compared) in order to investigate if specific nodes(genes) or gene areas are changing (and therefore can be considered affected by a treatment).

# Load Networks



In [2]:
#location where the raw data files are stored, it is set to run from the installation folder
#- if applicable please CHANGE or CHANGE to the location of your networks

graph_location = "../networks/edgelists/"

#location where output should be saved
#Please set location
location = ""


First step of the pipeline consists in loading the chosen data set.
You can store your networks in any common format, however the VOLTA package requires that the networks are provided as NetworkX Graph objects (refer to its documentation for detailed instructions). Moreover, the networks should be weighted: if you have an unweighted network, then assign all edges the same edge weight. The package assumes "weight" to be the default edge weight label, but this can be set when needed.

An example on how to pre-process a network, stored as an edgelist, is provided below. Different loading and storing examples are provided in the "import and export of networks" jupyter notebook. 

In [3]:
import glob
import pandas as pd
import networkx as nx
import numpy as np
import os

In [4]:

labels = []
networks_graphs = []
cnt = 0
print("load networks")
#gets all files located in the specified folder that end on .edgelist
#CHANGE the ending if your files end differently
for path in glob.glob(graph_location +"*.edgelist"):
    if "A549" in path:
        #you can specify that only part of the file name should be used as network name for later identification
        name =  path.split(os.path.sep)[-1].replace(".rds.edgelist", "")

        
        #read the edgelist file as a dataframe
        fh = pd.read_csv(path, sep="\t")
        #convert it into a NetworkX graph G and specify the column names of the node pairs
        G=nx.from_pandas_edgelist(fh, "V1", "V2")

        #if you have an unweighted network assign all edges the same edge weight - here a value of 1 is assigned
        for u, v, d in G.edges(data=True):
            d['weight'] = 1


        #save the graph objects to a list (only suitable if small networks are processed)
        #this is the main objects used for the examples below, which contains all networks
        networks_graphs.append(G)
        labels.append(name)




        print("loaded", name)
    cnt = cnt + 1

load networks
loaded dasatinib_A549
loaded mitoxantrone_A549


The networkX graph object is converted into a list of lists format

In [5]:
networks = volta.get_node_similarity.preprocess_graph(networks_graphs, attribute="weight")

Optional: If multiple networks are provided, get the union of nodes between them in order to ensure that all node names are mapped to the correct IDs (if this transformation is applied)

In [6]:
#get union of nodes

nodes = []
for net in networks_graphs:
    for node in net.nodes():
        if node not in nodes:
            nodes.append(node)

In [7]:
#mapp node names to ID (this is mainly used for node & edge similarity functions)

network_lists, mapping = volta.get_node_similarity.preprocess_node_list(networks)

In [8]:
#save mapping for later

import pickle

with open(location + "node_id_mapping_network_network.pckl", "wb") as f:
    pickle.dump(mapping, f, protocol=4)

In [9]:
#OPTIONAL: create reversed mapping object

reverse_mapping = volta.distances.node_edge_similarities.reverse_node_edge_mapping(mapping)

# Nodes

The networks are compared based on different centrality measures (how the node location in the network changes) ato estimate which nodes are the most similar or different w.r.t. their network position.

The centrality ranks are also used in the Network clustering pipeline.

In [10]:
#sort nodes after the selected attributes

sorted_nodes = []

for graph in networks_graphs:
    temp = volta.distances.node_edge_similarities.sort_node_list(graph, mapping, degree_centrality=True, closeness_centrality=True, betweenness=True, k=None, as_str=False)
    
    sorted_nodes.append(temp)

1
2
3
average position is calculated
1
2
3
average position is calculated


Below we convert the output, which is a Python dict into a dataframe to make it more human readable

In [11]:
mapping_ids = list(mapping.values())

In [12]:
import pandas as pd 

df = pd.DataFrame(mapping_ids, 
               columns =['Mapping ID']) 

In [13]:
# add the reversed mapping IDS (original node IDs - in the example networks they are Entrez IDs)
entrez = []
for g in mapping_ids:
    entrez.append(reverse_mapping[g])
df["Entrez IDs"] = entrez

In [14]:
for i in range(len(sorted_nodes)):
    item = sorted_nodes[i][0]
    for key in item.keys():
        #ignore "degree" key, since it has not been calculated. We are using degree centrality instead.
        #Adjust to your selected parameters
        if key != "degree":
            temp = []
            for g in mapping_ids:
                for xx in range(len(item[key])):
                    if item[key][xx] == g:
                        temp.append(xx)
                
            #add to dataframe
            #since the results are in the same order as the network labels 
            #we can use the network label directly as column heading
            df[labels[i]+" Ranking " + key] = temp


In [15]:
#display the dataframe

df

,Mapping ID,Entrez IDs,dasatinib_A549 Ranking dc,dasatinib_A549 Ranking cc,dasatinib_A549 Ranking betweenness,dasatinib_A549 Ranking average_mean,dasatinib_A549 Ranking average_median,mitoxantrone_A549 Ranking dc,mitoxantrone_A549 Ranking cc,mitoxantrone_A549 Ranking betweenness,mitoxantrone_A549 Ranking average_mean,mitoxantrone_A549 Ranking average_median
0,0,780,817,864,946,890,871,438,317,331,351,329
1,1,7538,207,168,125,149,163,235,224,292,234,233
2,2,9638,723,764,881,808,775,329,336,365,331,335
3,3,10954,308,263,287,276,283,673,710,600,668,677
4,4,6919,401,414,616,476,416,417,406,282,359,414
...,...,...,...,...,...,...,...,...,...,...,...,...
972,972,4891,923,925,904,927,929,699,822,479,673,704
973,973,29082,967,973,961,969,969,902,940,866,913,904
974,974,9870,936,947,928,943,941,570,477,271,446,482
975,975,5184,969,972,959,968,970,177,187,485,264,178


We are interested in knowing which genes change the most between the networks with regards to their network position. Therefore we are going to estimate the rank difference of the median ranks. 
This can be done for any of the other parameters as well, if it is needed for your analysis in the same way.Please refer to the functions documentation for more details

In [16]:
change = []

for g in mapping_ids:
    
    val1 = df.loc[df["Mapping ID"] == g][labels[0]+" Ranking average_median"].to_list()[0]
    
    val2 = df.loc[df["Mapping ID"] == g][labels[1]+" Ranking average_median"].to_list()[0]
    
    change.append(abs(val1-val2))  

In [17]:
df_change = pd.DataFrame(list(zip(mapping_ids, entrez, change, df[labels[0]+" Ranking average_median"].to_list(), df[labels[1]+" Ranking average_median"].to_list())), 
               columns =['Mapping ID', 'Entrez IDs', 'Absolute Ranking Difference', labels[0]+' Ranking average_median', labels[1]+' Ranking average_median' ]) 

In [18]:
#sort the dataframe for easier visualization/ analysis

df_change = df_change.sort_values(by =["Absolute Ranking Difference"], axis=0, ascending=False)

First we inspect the top 20 genes (which network position is the most affected between the compared networks). Adjust the value if need be.

In [19]:
df_change.head(20)

,Mapping ID,Entrez IDs,Absolute Ranking Difference,dasatinib_A549 Ranking average_median,mitoxantrone_A549 Ranking average_median
723,723,5018,924,964,40
965,965,10730,921,946,25
483,483,9817,895,19,914
702,702,4931,884,60,944
436,436,29103,876,927,51
449,449,54555,863,43,906
12,12,51097,858,96,954
504,504,1070,851,12,863
379,379,26136,839,128,967
624,624,29978,838,83,921


Then we inspect the bottom 20 genes (which network position changes the least between the two networks). Adjust the value if need be.

In [20]:
df_change.tail(20)

,Mapping ID,Entrez IDs,Absolute Ranking Difference,dasatinib_A549 Ranking average_median,mitoxantrone_A549 Ranking average_median
412,412,2771,8,631,639
638,638,29890,7,354,347
893,893,23223,7,891,884
679,679,55837,6,967,973
747,747,9738,6,724,718
215,215,28969,5,586,591
928,928,10270,5,719,724
277,277,29928,4,569,573
119,119,10491,4,461,465
297,297,388650,4,48,52


As a possible further analysis these ranked genes can be enriched by means of GSEA or the top x coulc be enriched as shown in the Example of Enrichment file.

# Edges

We now evaluate which edges are common in the two networks, which edges are unique and finally, which edges network position changes the most. The latter is estimated through betweenness estimation. 

In [21]:
#compute the edge betweenness scores and assign them to the graph objects

print("sort edges after edge betweenness")
bet = []
graphs_with_betweenness = []
for net in networks_graphs:
    edges_betweenness = nx.edge_betweenness_centrality(net)
    bet.append(edges_betweenness)
    #write as new attribute to graph
    temp = nx.set_edge_attributes(net, edges_betweenness, "betweenness")

sort edges after edge betweenness


As in the previous section the networks are converted to a list of list format and each edge is getting a unique ID assigned.

In [22]:
networks = volta.get_edge_similarity.preprocess_graph(networks_graphs, attribute="betweenness")

print("map edges to id")

network_lists, mapping = volta.get_edge_similarity.preprocess_edge_list(networks)

with open(location + "edge_id_mapping_network_network.pckl", "wb") as f:
    pickle.dump(mapping, f, protocol=4)

map edges to id


In [23]:
reverse_mapping = volta.distances.node_edge_similarities.reverse_node_edge_mapping(mapping)

The shared edges are retrieved. The function returns a dictionary data format, where key is mapped edge ID and value is list of network names this edge is present in.

In [24]:
shared = volta.distances.node_edge_similarities.compute_shared_layers(network_lists, labels, is_file=False, in_async=False)

dasatinib_A549
mitoxantrone_A549


The output is converted into a dataframe for easier inspection of the results.

In [25]:
edges = list(reverse_mapping.values())
edge_mapped_IDs = list(reverse_mapping.keys())

df = pd.DataFrame(list(zip(edges, edge_mapped_IDs)), 
               columns =['Edges', 'Mapping ID']) 
    

In [26]:
for label in labels:
    temp = []
    for i in edge_mapped_IDs:
        if label in shared[i]:
            temp.append(1)
        else:
            temp.append(0)
            
    df["In "+label] = temp
    

In [27]:
#plot the dataframe

df

,Edges,Mapping ID,In dasatinib_A549,In mitoxantrone_A549
0,"7538,780",0,1,0
1,"9638,780",1,1,0
2,"10954,780",2,1,0
3,"6919,780",3,1,0
4,"27333,780",4,1,0
...,...,...,...,...
25373,"23338,5257",25373,0,1
25374,"2274,26036",25374,0,1
25375,"23338,9943",25375,0,1
25376,"64080,23499",25376,0,1


The shared edges are retrieved and stored in a dataframe for inspection.

In [28]:
shared_df = df.loc[(df["In "+labels[0]] == 1) & (df["In "+labels[1]] == 1)]

In [29]:
shared_df

,Edges,Mapping ID,In dasatinib_A549,In mitoxantrone_A549
5,"9183,780",5,1,1
10,"9813,7538",10,1,1
20,"1459,7538",20,1,1
73,"23,6919",73,1,1
79,"27336,6919",79,1,1
...,...,...,...,...
10620,"8821,1154",10620,1,1
10634,"10123,10797",10634,1,1
10682,"5373,7840",10682,1,1
10790,"3628,2582",10790,1,1


The unique edges are retrieved and stored in a dataframe for inspection.

In [30]:
unique_df = df.loc[((df["In "+labels[0]] == 1) & (df["In "+labels[1]] == 0)) | ((df["In "+labels[0]] == 0) & (df["In "+labels[1]] == 1))]

In [31]:
unique_df

,Edges,Mapping ID,In dasatinib_A549,In mitoxantrone_A549
0,"7538,780",0,1,0
1,"9638,780",1,1,0
2,"10954,780",2,1,0
3,"6919,780",3,1,0
4,"27333,780",4,1,0
...,...,...,...,...
25373,"23338,5257",25373,0,1
25374,"2274,26036",25374,0,1
25375,"23338,9943",25375,0,1
25376,"64080,23499",25376,0,1


As possible further analysis the genes making up common or unique edges could be enriched as shown in the Example of Enrichment jupyter notebook.

# Node areas/ connectivity

Hereafter we evaluate wheter nodes are connected in a similar way among the two networks, as well as if there are differences among node areas. In order to do this, we will make use of the random walks method, as already shown in the network clustering example file. 

## Random walks

For each common node in the two networks, random walks are performed and their similarity in visited nodes is compared. This allows to identify the most similar/ dissimilar node areas.

For a subset of nodes, random walks of size 5 are performed by its degree number of times. A smaller walk size "scans" a smaller area around the starting node.Number of steps and number of walkers can be increased/decreased according to the experimental purposes (and memory availability). 
The number of nodes random walks are performed on can be adjusted as needed as well as the nodes on which they are performed on (e.g. the highest degree nodes of each network could be selected).


In [32]:
import random

nodess = random.sample(nodes, int(len(nodes)/10))

In [33]:
performed_walks = volta.get_walk_distances.helper_walks(networks_graphs, nodess, labels, steps=5, number_of_walks=1, degree=True, probabilistic=False, weight ="weight")

walks for node  0 outof 97
running walks 6 for node 9587
running walks 44 for node 9587
running walks 11 for node 9805
running walks 26 for node 9805
running walks 12 for node 6446
running walks 20 for node 6446
running walks 12 for node 166647
running walks 8 for node 166647
running walks 7 for node 11065
running walks 26 for node 11065
running walks 25 for node 10617
running walks 92 for node 10617
running walks 10 for node 8835
running walks 86 for node 8835
running walks 23 for node 5607
running walks 29 for node 5607
running walks 9 for node 8870
running walks 57 for node 8870
running walks 6 for node 10810
running walks 20 for node 10810
running walks 10 for node 5468
running walks 15 for node 5468
running walks 61 for node 10559
running walks 18 for node 10559
running walks 8 for node 64781
running walks 69 for node 64781
running walks 15 for node 2184
running walks 11 for node 2184
running walks 34 for node 10589
running walks 56 for node 10589
running walks 7 for node 6793
run

Now we are estimating for each starting node how often surrounding nodes/ edges have been visit with respect to all the visited nodes/ edges. Depending on your network sizes and selected nodes this can be quite memory intensive.

In [34]:
node_counts, edge_counts, nodes_frc, edges_frc = volta.get_walk_distances.helper_get_counts(labels, networks_graphs, performed_walks)

Now we want to estimate network similarities based on the visited nodes. For each network pair, kendall rank correlation is calculated (of the top 20 nodes; adjust this value as needed) for the same starting node. The mean correlation value of all same node pairs for a network pair is estimated as well as the individual values are calculated and returned.

In [35]:
combined_res = volta.get_walk_distances.helper_walk_sim(networks_graphs, performed_walks, nodess, labels, top=20, undirected=False, return_all = True, nodes_ranked=nodes_frc, edges_ranked=edges_frc)

The results are converted into a dataframe for inspection and the top and bottom 20 nodes are displayed.

In [40]:

df = pd.DataFrame(list(zip(nodess, combined_res["intermediate correaltion scores nodes"][(labels[0], labels[1])])), 
               columns =['Entrez ID', 'Correlation']) 

#dataframe is sorted after correlation

df = df.sort_values(by =["Correlation"], axis=0, ascending=False)

In [41]:
df.head(20)

,Entrez ID,Correlation
34,207,0.400000
80,55893,0.400000
87,79090,0.336842
76,29940,0.315789
57,1515,0.305263
48,8727,0.294737
74,3978,0.273684
29,55152,0.273684
4,11065,0.263158
73,4616,0.263158


In [42]:
df.tail(20)

,Entrez ID,Correlation
96,3280,-0.094737
43,51805,-0.094737
44,2810,-0.094737
45,8914,-0.105263
1,9805,-0.115789
16,2073,-0.147368
69,23483,-0.157895
60,7126,-0.168421
84,6697,-0.168421
79,9246,-0.189474


Additionally the networks can be compared by means of their community structure as shown in the community notebook file.